In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout, Conv1D, MaxPooling1D, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical


2025-10-06 03:13:43.303840: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from KeplerModel import KeplerModel

In [3]:
""" 
Example of an artificial neural network model
Derived from class KerasModel
"""
class MLPModel(KeplerModel):
  def __init__(self, modelname, hparam):
    super().__init__(modelname, hparam)

    # Construct the model
    inputShape = (2,256)
    inputs = Input(shape=inputShape)
    self.BuildModel(inputs, hparam)

    self.SetLearningRate(hparam['learning_rate'])
    self.SetLoss('mae')
    self.SetMetrics(['mae','mse'])
    self.SetOptimizer(Adam)
    #self.Compile()
  
    self.SetModelName("mlp")
    self.SetModelVersion(1)
    self.SetModelTrial(1)
    
    
  def MLP(self, inputs, hp):
    # Construct an MLP model

    m = inputs
    
    self.dropout=np.full((hp['layers']),hp['dropout'])
    batchNorm = hp['batch_norm']
    act = hp['activation']
    units = hp['units']
    layers = hp['layers']
    expansion = hp['expansion']
    for layer in range(layers):
      m = Dense(units)(m)
      if batchNorm:
        m = BatchNormalization()(m)
      m = activations.get(act)(m)
      if not batchNorm and self.dropout[layer] > 0.0:
        m = Dropout(self.dropout[layer])(m)
      units *= expansion

    m = Dense(hp['final_units'])(m)
    m = activations.get(hp['final_activation'])(m)
    if batchNorm:
      m = BatchNormalization()(m)
    if not batchNorm and hp['final_dropout'] > 0.0:
      m = Dropout(hp['final_dropout'])(m)

    m = Dense(32, activation=hp['final_activation'])(m)
    target = Dense(1, activation=hp['target_activation'], name='target')(m)
    
    return target
    
  def BuildModel(self, inputs, hp):
    # Construct the model

    target = self.MLP(inputs, hp)
    m = Model(inputs=inputs, outputs=[target], name="mlp")
    self.SetModel(m)
    return m
    






In [4]:
def main():
  hyperParam = {
    'version': 1,
    'revision': 0,
    'trial': 1,
    'units': 1024,
    'expansion': 0.5,
    'layers': 6,
    'activation': 'relu',
    'batch_norm': True,
    'dropout': 0.0,
    'final_units': 32,
    'final_dropout': 0.0,
    'final_activation': 'relu',
    'target_activation': 'relu',
    'learning_rate': 0.001,
    'batch_size': 64,
    'epoch': 0,
    'begin': 0,
    'epochs': 10,
    'patience': 0,
    'threshold': 0,
  }

  m = MLPModel('mlp', hyperParam)
  print(m.model.summary())

  m.SetModelTrial(1)
  m.SetModelRevision(1)
  m.SetModelPath("./model")
  m.SaveModel()

In [5]:
# -
if __name__ == '__main__':
  main()


2025-10-06 03:13:44.985595: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-10-06 03:13:44.985623: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: nvgpu
2025-10-06 03:13:44.985631: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: nvgpu
2025-10-06 03:13:44.985692: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.163.1
2025-10-06 03:13:44.985711: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 550.163.1
2025-10-06 03:13:44.985719: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 550.163.1


Model: "mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 256)]          0         
                                                                 
 dense (Dense)               (None, 2, 1024)           263168    
                                                                 
 batch_normalization (Batch  (None, 2, 1024)           4096      
 Normalization)                                                  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 2, 1024)           0         
                                                                 
 dense_1 (Dense)             (None, 2, 512)            524800    
                                                                 
 batch_normalization_1 (Bat  (None, 2, 512)            2048      
 chNormalization)                                              

INFO:tensorflow:Assets written to: model/mlpv1r1t1-e0/assets


In [7]:
!ls -lt model

total 0
drwxr-xr-x 4 root root 106 Oct  6 03:13 mlpv1r1t1-e0
